In [1]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
from pymongo import GEOSPHERE
import seaborn as sns
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from dotenv import load_dotenv
import os
from functools import reduce
import operator
load_dotenv()

True

In [2]:
### FOURSQUARE
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [3]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")


In [66]:
companies = db.get_collection("companies")
gaming_proj = {"_id": 0, "name": 1, "offices.latitude": 1, "offices.longitude": 1, "number_of_employees": 1}
gaming_companies = list(companies.find({"number_of_employees": {"$gt": 50}, "total_money_raised": {"$gte": "$1M"}, "offices.zip_code": {"$exists": True}, "offices.city": {"$exists": True}}, gaming_proj))
gaming_companies

[{'name': 'Digg',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.764726, 'longitude': -122.394523}]},
 {'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': [{'latitude': 37.41605, 'longitude': -122.151801},
   {'latitude': 53.344104, 'longitude': -6.267494},
   {'latitude': 40.7557162, 'longitude': -73.9792469}]},
 {'name': 'eBay',
  'number_of_employees': 15000,
  'offices': [{'latitude': 37.295005, 'longitude': -121.930035}]},
 {'name': 'Cisco',
  'number_of_employees': 63000,
  'offices': [{'latitude': 37.408802, 'longitude': -121.95377}]},
 {'name': 'Yahoo!',
  'number_of_employees': 13600,
  'offices': [{'latitude': 37.418531, 'longitude': -122.025485}]},
 {'name': 'Powerset',
  'number_of_employees': 60,
  'offices': [{'latitude': 37.778613, 'longitude': -122.395289}]},
 {'name': 'AddThis',
  'number_of_employees': 120,
  'offices': [{'latitude': 38.926172, 'longitude': -77.245195},
   {'latitude': 40.724604, 'longitude': -73.996876},
   {'latitude': 34.02

In [6]:
gaming_companies = db.get_collection("gaming_companies")
la_companies = list(gaming_companies.find())
la_companies
los_angeles = pd.DataFrame(la_companies)
latitude = [lat["offices"][0]["latitude"] for lat in la_companies]
longitude = [long["offices"][0]["longitude"] for long in la_companies]
zip_code = [code["offices"][0]["zip_code"] for code in la_companies]
address = [point["offices"][0]["address1"] for point in la_companies]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
los_angeles["latitude"] = latitude
los_angeles["longitude"] = longitude
los_angeles["zip_code"] = zip_code
los_angeles["address"] = address

los_angeles = los_angeles[los_angeles["latitude"].notna()]
los_angeles = los_angeles[los_angeles["longitude"].notna()]

los_angeles.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,milestones,video_embeds,screenshots,external_links,partners,ipo,latitude,longitude,zip_code,address
0,52cdef7c4bab8bd675297d8d,Digg,digg,http://www.crunchbase.com/company/digg,http://www.digg.com,http://blog.digg.com/,http://blog.digg.com/?feed=rss2,digg,news,60.0,...,"[{'id': 9588, 'description': 'Another Digg Exe...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[117, 150], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[],NaN,37.764726,-122.394523,94107,135 Mississippi St
1,52cdef7c4bab8bd675297d94,Twitter,twitter,http://www.crunchbase.com/company/twitter,http://twitter.com,http://twitter.com/blog,http://feeds.feedburner.com/TwitterBlog,twitter,social,1300.0,...,"[{'id': 1, 'description': 'Survives iPhone 3G ...","[{'embed_code': '<iframe width=""430"" height=""2...","[{'available_sizes': [[[150, 74], 'assets/imag...",[{'external_url': 'http://sec.gov/cgi-bin/brow...,[],"{'valuation_amount': 18100000000, 'valuation_c...",37.776805,-122.416924,94103,1355 Market St.
2,52cdef7c4bab8bd675297d95,StumbleUpon,stumbleupon,http://www.crunchbase.com/company/stumbleupon,http://www.stumbleupon.com,http://www.stumbleupon.com/blog,,stumbleupon,web,NaN,...,"[{'id': 3103, 'description': 'Spin-out from Eb...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.facebook.com/Stu...,[],NaN,37.775196,-122.419204,,
3,52cdef7c4bab8bd675297d97,Scribd,scribd,http://www.crunchbase.com/company/scribd,http://scribd.com,http://blog.scribd.com/,http://blog.scribd.com/feed/,scribd,news,50.0,...,"[{'id': 4960, 'description': 'Scribd launches ...","[{'embed_code': '<script src=""http://i.cdn.tur...","[{'available_sizes': [[[150, 72], 'assets/imag...",[{'external_url': 'http://www.online-tech-tips...,[],NaN,37.789634,-122.404052,94107,539 Bryant Street
4,52cdef7c4bab8bd675297da4,Powerset,powerset,http://www.crunchbase.com/company/powerset,http://powerset.com,http://blog.powerset.com/,http://blog.powerset.com/atom.xml,Powerset,search,60.0,...,[],"[{'embed_code': '<object width=""400"" height=""3...",[],[{'external_url': 'http://en.wikipedia.org/wik...,[],NaN,37.778613,-122.395289,94107,475 Brannan St


In [7]:
los_angeles.groupby("category_code").count().sort_values(by = "_id", ascending = False).head(10)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,number_of_employees,founded_year,...,milestones,video_embeds,screenshots,external_links,partners,ipo,latitude,longitude,zip_code,address
category_code,,,,,,,,,,,,,,,,,,,,,
web,189,189,189,189,189,189,188,145,105,159,...,189,189,189,189,189,2,189,189,185,184
software,94,94,94,94,94,94,94,80,53,82,...,94,94,94,94,94,3,94,94,92,92
advertising,70,70,70,70,70,70,69,56,40,61,...,70,70,70,70,70,2,70,70,70,70
games_video,51,51,51,51,51,51,50,43,34,48,...,51,51,51,51,51,2,51,51,50,50
mobile,48,48,48,48,48,48,48,36,28,41,...,48,48,48,48,48,1,48,48,47,47
enterprise,40,40,40,40,40,40,40,37,28,39,...,40,40,40,40,40,0,40,40,40,40
ecommerce,23,23,23,23,23,23,23,19,18,19,...,23,23,23,23,23,1,23,23,23,23
consulting,21,21,21,21,21,21,21,14,18,19,...,21,21,21,21,21,0,21,21,21,21
network_hosting,21,21,21,21,21,21,21,11,8,17,...,21,21,21,21,21,0,21,21,20,20


In [17]:
#Geoquery inicial

la = {'type': 'Point', 'coordinates': [34.0194, -118.411]}
url_query = 'https://api.foursquare.com/v2/venues/explore'

parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{sf.get('coordinates')[1]}",
    "query": "basketball", 
    "limit": 100    
}

In [18]:
resp = requests.get(url= url_query, params = parametros).json()
data = resp.get("response").get("groups")[0].get("items")

In [19]:
#Vamos a sacar el primer DF
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lng"]
mapa_longitud = ["venue", "location", "lat"]

def getFromDict(diccionario,mapa):
    
    return reduce(operator.getitem,mapa,diccionario)

basket_list = []
for dic in data:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    basket_list.append(paralista)
    

documents = []
for diccionario in basket_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
basket_venues = pd.DataFrame(documents)
basket_venues["place"] = "basketball court"
basket_venues.head()

,name,location,place
0,Mar Vista Recreation Center And Park,"{'type': 'Point', 'coordinates': [-118.4285552...",basketball court
1,Culver City Veterans' Memorial Center,"{'type': 'Point', 'coordinates': [-118.4024548...",basketball court
2,Cheviot Hills Park,"{'type': 'Point', 'coordinates': [-118.4096145...",basketball court
3,Glen Alla Park & Recreation,"{'type': 'Point', 'coordinates': [-118.4316127...",basketball court
4,Honey Training Center,"{'type': 'Point', 'coordinates': [-118.3997953...",basketball court


In [13]:
basket_venues.to_json("basket_venues_la.json", orient="records")

In [15]:
latitude = [lat["latitud"] for lat in basket_list]
longitude = [long["longitud"] for long in basket_list]

In [16]:
basket_venues["latitude"] = longitude
basket_venues["longitude"] = latitude
basket_venues

,name,location,place,latitude,longitude
0,Mar Vista Recreation Center And Park,"{'type': 'Point', 'coordinates': [-118.4285552...",basketball court,34.017516,-118.428555
1,Culver City Veterans' Memorial Center,"{'type': 'Point', 'coordinates': [-118.4024548...",basketball court,34.012783,-118.402455
2,Cheviot Hills Park,"{'type': 'Point', 'coordinates': [-118.4096145...",basketball court,34.046010,-118.409615
3,Glen Alla Park & Recreation,"{'type': 'Point', 'coordinates': [-118.4316127...",basketball court,33.984516,-118.431613
4,Honey Training Center,"{'type': 'Point', 'coordinates': [-118.3997953...",basketball court,33.983233,-118.399795
...,...,...,...,...,...
72,Staples Center Suite A-31,"{'type': 'Point', 'coordinates': [-118.2665863...",basketball court,34.047635,-118.266586
73,Lob City,"{'type': 'Point', 'coordinates': [-118.2651703...",basketball court,34.046353,-118.265170
74,Staple Ctr Patio S,"{'type': 'Point', 'coordinates': [-118.2604196...",basketball court,34.051451,-118.260420
75,Dont Know,"{'type': 'Point', 'coordinates': [-118.2592661...",basketball court,34.050561,-118.259266


In [22]:
#Sacamos vegan_venues.

parametros_1 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "vegan", 
    "limit": 100    
}

In [23]:
resp_1 = requests.get(url= url_query, params = parametros_1).json()
data_1 = resp_1.get("response").get("groups")[0].get("items")
data_1

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5228e77411d2096e343a78f9',
   'name': 'Sage Organic Vegan Bistro',
   'contact': {},
   'location': {'address': '4130 Sepulveda Blvd',
    'crossStreet': 'at Washington Blvd',
    'lat': 34.00640912972718,
    'lng': -118.41218139178056,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.00640912972718,
      'lng': -118.41218139178056}],
    'distance': 1450,
    'postalCode': '90230',
    'cc': 'US',
    'neighborhood': 'West Los Angeles',
    'city': 'Culver City',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['4130 Sepulveda Blvd (at Washington Blvd)',
     'Culver City, CA 90230',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1d3941735',
     'name': 'Vegetarian / Vegan Restaurant',
     'pluralName': 'Vegetarian / Vegan Restaurants',
     'shortName': 'Vegeta

In [25]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

vegan_list = []
for dic in data_1:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    vegan_list.append(paralista)
    

documents = []
for diccionario in vegan_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
vegan_venues = pd.DataFrame(documents)
vegan_venues["place"] = "vegan"
vegan_venues.head()

,name,location,place
0,Sage Organic Vegan Bistro,"{'type': 'Point', 'coordinates': [-118.4121813...",vegan
1,Mayura Indian Restaurant,"{'type': 'Point', 'coordinates': [-118.4040374...",vegan
2,Tender Greens,"{'type': 'Point', 'coordinates': [-118.3952543...",vegan
3,The Vegan Joint,"{'type': 'Point', 'coordinates': [-118.4116094...",vegan
4,Samosa House East,"{'type': 'Point', 'coordinates': [-118.405493,...",vegan


In [26]:
vegan_venues.to_json("vegan_venues_la.json", orient="records")

In [27]:
parametros_2 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "starbucks", 
    "limit": 100    
}

In [28]:
resp_2 = requests.get(url= url_query, params = parametros_2).json()
data_2 = resp_2.get("response").get("groups")[0].get("items")
data_2

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5c8837da1f8ed6002c917d56',
   'name': 'Starbucks',
   'contact': {},
   'location': {'address': '10601 W Washington Blvd',
    'lat': 34.017179,
    'lng': -118.405394,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.017179,
      'lng': -118.405394}],
    'distance': 573,
    'postalCode': '90232',
    'cc': 'US',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['10601 W Washington Blvd',
     'Los Angeles, CA 90232',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tip

In [31]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

starbucks_list = []
for dic in data_2:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    starbucks_list.append(paralista)
    

documents = []
for diccionario in starbucks_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
starbucks_venues = pd.DataFrame(documents)
starbucks_venues["place"] = "starbucks"
starbucks_venues.head()

,name,location,place
0,Starbucks,"{'type': 'Point', 'coordinates': [-118.405394,...",starbucks
1,Starbucks,"{'type': 'Point', 'coordinates': [-118.42226, ...",starbucks
2,Starbucks,"{'type': 'Point', 'coordinates': [-118.3961440...",starbucks
3,Starbucks,"{'type': 'Point', 'coordinates': [-118.41255, ...",starbucks
4,Starbucks,"{'type': 'Point', 'coordinates': [-118.4277146...",starbucks


In [30]:
starbucks_venues.to_json("starbucks_venues_la.json", orient="records")

In [32]:
parametros_3 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "school", 
    "limit": 100    
}

In [33]:
resp_3 = requests.get(url= url_query, params = parametros_3).json()
data_3 = resp_3.get("response").get("groups")[0].get("items")
data_3

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bc36695f8219c741c09b610',
   'name': 'Gymboree Play & Music, Westside Pavilion',
   'contact': {},
   'location': {'address': '10800 W Pico Blvd #207',
    'lat': 34.0401391,
    'lng': -118.4274689,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.0401391,
      'lng': -118.4274689}],
    'distance': 2763,
    'postalCode': '90064',
    'cc': 'US',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['10800 W Pico Blvd #207',
     'Los Angeles, CA 90064',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d13b941735',
     'name': 'School',
     'pluralName': 'Schools',
     'shortName': 'School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'verified': True,

In [34]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

school_list = []
for dic in data_3:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    school_list.append(paralista)
    

documents = []
for diccionario in school_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
school_venues = pd.DataFrame(documents)
school_venues["place"] = "school"
school_venues.head()

,name,location,place
0,"Gymboree Play & Music, Westside Pavilion","{'type': 'Point', 'coordinates': [-118.4274689...",school
1,New World Montessori,"{'type': 'Point', 'coordinates': [-118.407486,...",school
2,Palms Middle School,"{'type': 'Point', 'coordinates': [-118.4157027...",school
3,Blue Pulse CPR,"{'type': 'Point', 'coordinates': [-118.4022116...",school
4,Echo Art Studio,"{'type': 'Point', 'coordinates': [-118.4100529...",school


In [35]:
school_venues.to_json("school_venues_la.json", orient="records")

In [36]:
parametros_4 = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "airport", 
    "limit": 100    
}

In [37]:
resp_4 = requests.get(url= url_query, params = parametros_4).json()
data_4 = resp_4.get("response").get("groups")[0].get("items")
data_4

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '439ec330f964a520102c1fe3',
   'name': 'Los Angeles International Airport (LAX) (Los Angeles International Airport)',
   'contact': {},
   'location': {'address': '1 World Way',
    'lat': 33.94368036522437,
    'lng': -118.40607404708864,
    'labeledLatLngs': [{'label': 'display',
      'lat': 33.94368036522437,
      'lng': -118.40607404708864}],
    'distance': 8441,
    'postalCode': '90045',
    'cc': 'US',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['1 World Way',
     'Los Angeles, CA 90045',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1ed931735',
     'name': 'Airport',
     'pluralName': 'Airports',
     'shortName': 'Airport',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'},
 

In [38]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

airport_list = []
for dic in data_4:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    airport_list.append(paralista)
    

documents = []
for diccionario in airport_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
airport_venues = pd.DataFrame(documents)
airport_venues["place"] = "airport"
airport_venues.head()

,name,location,place
0,Los Angeles International Airport (LAX) (Los A...,"{'type': 'Point', 'coordinates': [-118.4060740...",airport
1,Santa Monica Airport (SMO) (Santa Monica Airport),"{'type': 'Point', 'coordinates': [-118.4476068...",airport
2,In an Uber,"{'type': 'Point', 'coordinates': [-118.4144647...",airport
3,LAX - Virgin Atlantic - Terminal 2,"{'type': 'Point', 'coordinates': [-118.4050172...",airport
4,Avis Car Rental,"{'type': 'Point', 'coordinates': [-118.410201,...",airport


In [39]:
airport_venues.to_json("airport_venues_la.json", orient="records")

In [40]:
parametros_5 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "pet hairdresser", 
    "limit": 100    
}

In [41]:
resp_5 = requests.get(url= url_query, params = parametros_5).json()
data_5 = resp_5.get("response").get("groups")[0].get("items")
data_5

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4ba39961f964a520a44a38e3',
   'name': 'Global Pet Food Outlet',
   'contact': {},
   'location': {'address': '4449 Sepulveda Blvd',
    'crossStreet': 'at Braddock Dr',
    'lat': 34.00244798606487,
    'lng': -118.40635268315941,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.00244798606487,
      'lng': -118.40635268315941}],
    'distance': 1935,
    'postalCode': '90230',
    'cc': 'US',
    'city': 'Culver City',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['4449 Sepulveda Blvd (at Braddock Dr)',
     'Culver City, CA 90230',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d100951735',
     'name': 'Pet Store',
     'pluralName': 'Pet Stores',
     'shortName': 'Pet Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
  

In [42]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

dog_list = []
for dic in data_5:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    dog_list.append(paralista)
    

documents = []
for diccionario in dog_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
dog_venues = pd.DataFrame(documents)
dog_venues["place"] = "dog"
dog_venues.head()

,name,location,place
0,Global Pet Food Outlet,"{'type': 'Point', 'coordinates': [-118.4063526...",dog
1,Centinela Feed & Pet Supply,"{'type': 'Point', 'coordinates': [-118.3971582...",dog
2,Centinela Feed & Pet Supplies,"{'type': 'Point', 'coordinates': [-118.4341513...",dog
3,Centinela Feed & Pet Supply,"{'type': 'Point', 'coordinates': [-118.4320314...",dog
4,PetSmart,"{'type': 'Point', 'coordinates': [-118.394868,...",dog


In [43]:
dog_venues.to_json("dog_venues_la.json", orient="records")

In [44]:
parametros_6 = {"client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{la.get('coordinates')[0]},{la.get('coordinates')[1]}",
    "query": "club", 
    "limit": 100    
}

In [45]:
resp_6 = requests.get(url= url_query, params = parametros_6).json()
data_6 = resp_6.get("response").get("groups")[0].get("items")
data_6

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4a1f723cf964a5202a7c1fe3',
   'name': 'Cafe 70°',
   'contact': {},
   'location': {'address': '3863 Overland Ave',
    'lat': 34.016919824076,
    'lng': -118.40640772789246,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.016919824076,
      'lng': -118.40640772789246}],
    'distance': 505,
    'postalCode': '90232',
    'cc': 'US',
    'city': 'Culver City',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['3863 Overland Ave',
     'Culver City, CA 90232',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
  

In [47]:
def getFromDict(diccionario,mapa):
    mapa_nombre =  ["venue", "name"]
    mapa_latitud = ["venue", "location", "lng"]
    mapa_longitud = ["venue", "location", "lat"]
    return reduce(operator.getitem,mapa,diccionario)

club_list = []
for dic in data_6:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    club_list.append(paralista)
    

documents = []
for diccionario in club_list:
    temporal = {
        "name": diccionario.get("name"),
        "location": {"type": "Point", "coordinates": [diccionario.get("latitud"), diccionario.get("longitud")]},
        }
    documents.append(temporal)
club_venues = pd.DataFrame(documents)
club_venues["place"] = "club"
club_venues.head()

,name,location,place
0,Cafe 70°,"{'type': 'Point', 'coordinates': [-118.4064077...",club
1,The Motor Club,"{'type': 'Point', 'coordinates': [-118.409711,...",club
2,The Griffin Club,"{'type': 'Point', 'coordinates': [-118.4097856...",club
3,Rasselbock Kitchen & Beer Garden,"{'type': 'Point', 'coordinates': [-118.430786,...",club
4,Sony Pictures Athletic Club,"{'type': 'Point', 'coordinates': [-118.4000859...",club


In [49]:
club_venues.to_json("club_venues_la.json", orient="records")

In [86]:
la_venues = db.get_collection("la_venues")

In [110]:
la_coordinates = [-118.387511, 33.902696] 
coord_point = {"type":"Point", "coordinates": la_coordinates}

In [119]:
query = {"location": {"$near": {"$geometry": coord_point,"$minDistance": 0  , "$maxDistance": 5000}}}

In [124]:
query_final = list(la_venues.find(query))

In [121]:
la_final = pd.DataFrame(query_final)
len(la_final)

14

In [122]:
latitude = [lat["location"]["coordinates"][1] for lat in query_final]
longitude = [long["location"]["coordinates"][0] for long in query_final]

#De aquí añadimos las listas que nos interesen al DataFrame de la ciudad de San Francisco.
la_final["latitude"] = latitude
la_final["longitude"] = longitude


la_final = la_final[la_final["latitude"].notna()]
la_final = la_final[la_final["longitude"].notna()]

la_final_companies = la_final[la_final["place"] == "company"]
la_final_companies

,_id,name,location,place,latitude,longitude
0,607db3d1202adb1ce64ceaca,Vantage Media,"{'type': 'Point', 'coordinates': [-118.387511,...",company,33.902696,-118.387511
1,607db3d1202adb1ce64cea64,iRise,"{'type': 'Point', 'coordinates': [-118.3812173...",company,33.901981,-118.381217
3,607db3d1202adb1ce64cea77,Velocify,"{'type': 'Point', 'coordinates': [-118.3952969...",company,33.919258,-118.395297


In [123]:
la_map = folium.Map(location = [33.902696, -118.387511], zoom_start = 15)
for i, row in la_final_companies.iterrows():
    distrito = {
        "location":[row["latitude"], row["longitude"]],
        "tooltip": [row["name"], "aquí tienes tus putos sitios, Pablo"]}
    
    icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "map-pin",
                    icon_color = "black")
        
        
    
    Marker(**distrito,icon = icon ).add_to(la_map)
la_map      